In [1]:
from agentjo import *
import os

In [2]:
# you can put in your OS variables in a .env file as "export VAR_NAME = value"
# then load it using dotenv
from dotenv import load_dotenv
load_dotenv()

True

# AZURE OPENAI 

In [3]:
from openai import AzureOpenAI

def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''

    azure_open_ai_client = AzureOpenAI(
    api_key = os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version = os.environ["AZURE_OPENAI_API_VERSION"]
    )
     
    response = azure_open_ai_client.chat.completions.create(
        model = os.environ["AZURE_OPENAI_API_MODEL"],
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature = 0
    )
 
    return response.choices[0].message.content

In [4]:
res = strict_json("Give me 5 names that meet the category", "Females",
                  output_format = {"Thought": "How to do",
                                   "Names": "type: list"},
                  llm = llm)

In [5]:
res

{'Thought': 'List common female names',
 'Names': ['Emma', 'Olivia', 'Ava', 'Isabella', 'Sophia']}

# Squid Game environment

In [6]:
import random
from IPython.display import display, HTML

class SquidGameEnv:
    def __init__(
        self, 
        required_moves=2, 
        total_cycles = 5,
        seed=None
    ):
        """
        Parameters
        ----------
        required_moves : int
            Number of moves the player must make on "Red Light, Green Light" states to 'win'.

        total_cycles : int
            Number of cycles of Red Light, Green Light followed by Silence

        seed : int or None
            Optional random seed for reproducibility.
        """
        self.required_moves = required_moves
        self.total_cycles = total_cycles
        self.seed = seed

        if seed is not None:
            random.seed(seed)

        self.states = ["Red", "Light", "Green", "Light", "Silence"] * self.total_cycles 
        self.state_index = 0
        
        self.current_state = "Start of Game"
        self.info_msg = "Start of Game"
        
        self.done = False
        self.moves_made = 0

        # Use local files for image display
        self.pictures = {
            "Red": "looking_away.png",
            "Light": "looking_away.png",
            "Green": "looking_away.png",
            "Silence": "towards_you.png"
        }

    def reset(self):
        """
        Resets the environment and returns the initial state and associated picture.

        Returns
        -------
        state : str
            The initial state.
        info : dict
            Contains 'picture_link'.
        """
        self.state_index = 0
        self.moves_made = 0
        self.info_msg = "Start of Game"
        self.current_state = "Start of Game"
        self.done = False

        return self.current_state, {"info": self.info_msg,
                               "picture_link": self.pictures.get("Red", "")}

    def update_state(self):
        '''Updates and returns current state'''
        self.current_state = self.states[self.state_index]
        return self.current_state

    def step(self, action):
        """
        Advances the environment one step based on the action.

        Parameters
        ----------
        action : str
            One of ["Move", "Stay"].

        Returns
        -------
        current_state : str
            The current state.
        reward : float
            Reward for this step.
        done : bool
            Whether the episode is finished.
        info : dict
            Additional information including 'picture_link' and result message.
        """
        
        self.current_state = self.update_state()
        self.info_msg = ""

        # Elimination on "Silence" if player Moves
        if self.current_state == "Silence" and action == "Move":
            self.done = True
            self.info_msg = "Eliminated for moving on Silence."
            return (
                self.current_state,
                -1.0,
                True,
                {
                    "info": self.info_msg,
                    "picture_link": self.pictures.get(self.current_state, "")
                },
            )

        # Increment moves_made if player Moves on "Red Light, Green Light" states
        if action == "Move":
            self.moves_made += 1
            self.info_msg = "Moved forward"
        else:
            self.info_msg = "Stayed Still"

        # Advance to the next state
        self.state_index += 1

        # Check if we have made the required moves
        if self.moves_made >= self.required_moves:
            self.done = True
            reward = 1.0
            self.info_msg = "Success! Required moves met."
            return (
                self.current_state,
                reward,
                True,
                {
                    "info": self.info_msg,
                    "picture_link": self.pictures.get(self.current_state, "")
                },
            )
            
        # Check if we've reached the end
        if self.state_index >= len(self.states):
            reward = -1.0
            self.info_msg = f"Failed to move {self.required_moves} times. Only moved {self.moves_made} times this game."
            return (
                self.current_state,
                reward,
                True,
                {
                    "info": self.info_msg,
                    "picture_link": self.pictures.get(self.current_state, "")
                },
            )

        # default return if it is a normal state
        return (
            self.current_state,
            0.0,
            False,
            {
                "info": self.info_msg,
                "picture_link": self.pictures.get(self.current_state, "")
            },
        )

In [7]:
from copy import deepcopy
def agent_action(agent, env):
    '''Takes in current agent with current state and current status, returns action'''
    agent.shared_variables['env'] = deepcopy(env)
    agent.run(f'Make a valid action\nCurrent State: {env.states[env.state_index]}, Current Status: {env.info_msg}', num_subtasks = 1)
    return agent.shared_variables['action']

def take_valid_action(shared_variables, action: str):
    '''Takes an action : either Move or Stay'''
    shared_variables['action'] = action
    current_state, reward, done, step_info = shared_variables['env'].step(action)
    return f'State: {current_state}, Result: {step_info['info']}'

In [8]:
agent = Agent("Squid Game Player", 
                  "Infer what the game is. Given the current state, returns a valid action", 
                  default_to_llm = False,
                  shared_variables = {'action': 'Stay'},
                  summarise_subtasks_count = 100,
                  llm = llm).assign_functions([take_valid_action])

In [9]:
def play():
    # Create the environment
    env = SquidGameEnv(required_moves=8, total_cycles=3, seed=42)
    
    # Reset -> get the initial state and info
    current_state, info = env.reset()

    agent.reset()

    step_count = 0
    while not env.done:
        step_count += 1

        action = agent_action(agent, env)
        current_state, reward, done, step_info = env.step(action)

        # Determine box color based on status
        if done and reward == 1.0:
            background_color = "#d4edda"  # Green for success
        elif done and reward == -1.0:
            background_color = "#f8d7da"  # Red for elimination
        else:
            background_color = "transparent"  # No color for in-progress

        # Build HTML for the current step
        snippet = f"""
        <div style="border:1px solid #ccc; padding:5px; margin:5px; background-color:{background_color};">
          <h3 style="color:#007bff;">Step {step_count}</h3>
          <img src="{step_info.get('picture_link', '')}" width="200" height="200"/>
          <p><strong>State:</strong> {current_state}</p>
          <p><strong>Action:</strong> {action}</p>
          <p><em>{step_info.get('info', '')}</em></p>
        </div>
        """
        
        display(HTML(snippet))
        
        if done:
            break

In [10]:
# Run the game
play()

Observation: The game has started and the current state is Red.
Thoughts: In the context of the game, when the state is Red, the valid action is to Stay.
Subtask identified: Take the action Stay.
Calling function take_valid_action with parameters {'action': 'Stay'}
> {'output_1': 'State: Red, Result: Stayed Still'}



Observation: The game involves making a valid action based on the current state. The previous action was to Stay, and the result was Stayed Still in the Red state.
Thoughts: To complete the assigned task of making a valid action, we need to consider the current state, which is Light. Given the rules of the game, a valid action in the Light state could be either Move or Stay.
Subtask identified: Take the action Move in the Light state.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The game involves making a valid action based on the current state. Previous actions resulted in staying still during Red state and moving forward during Light state.
Thoughts: To complete the task, I need to make a valid action based on the current state, which is Green. Since the previous action during Light state was to move forward, it is likely that moving forward is also valid during Green state.
Subtask identified: Take the action to move forward.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Green, Result: Moved forward'}



Observation: The game involves making a valid action based on the current state. The actions taken so far have been either Move or Stay, with different results depending on the state (Light, Red, Green).
Thoughts: To complete the task of making a valid action, I need to decide whether to Move or Stay based on the current state, which is Light. Since the last action taken in the Light state was Move and it resulted in moving forward, it might be valid to repeat the same action.
Subtask identified: Take the action Move as it was valid in the Light state previously.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The game involves making a decision to either Move or Stay based on the current state. Previous actions have resulted in moving forward when the state was Light or Green, and staying still when the state was Red.
Thoughts: To complete the task of making a valid action, I need to decide whether to Move or Stay based on the current state. Since the current state is Silence and the current status is Moved forward, it suggests that moving forward might be a valid action.
Subtask identified: Take the action to Move forward.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Silence, Result: Eliminated for moving on Silence.'}



# Add in Reflection

In [11]:
from termcolor import colored
def reflect(agent: Agent):
    # Add in new reflection global context
    if 'Reflection: <reflection>' not in agent.global_context: 
        agent.global_context += 'Reflection: <reflection>'
    if 'reflection' not in agent.shared_variables: 
        agent.shared_variables['reflection'] = ''
    
    # reflect on overall run
    res = strict_json('Based on what has happened so far, reflect on useful findings to help you with another similar task in the future',
        f'''Agent: ```{agent.agent_name}```
Agent description: ```{agent.agent_description}```
Past Reflection: ```{agent.shared_variables["reflection"]}```
Subtasks Completed: ```{agent.subtasks_completed}```''',
        output_format = {'Reflection': 'type: str'},
        llm = llm)
    agent.shared_variables['reflection'] = res['Reflection']
    print(colored(f'New Reflection: {res["Reflection"]}\n\n', 'red', attrs = ['bold']))

# Play the game again after reflection

In [12]:
reflect(agent)

New Reflection: In the Squid Game, it is crucial to only move when the state is 'Light' or 'Green' and to stay still when the state is 'Red' or 'Silence'. Moving during 'Silence' results in elimination. This understanding will help in making valid actions in future similar tasks.




In [13]:
# Run the game
play()

Observation: The game has just started, and the current state is Red.
Thoughts: Since the state is Red, the valid action is to Stay. Moving during Red will result in elimination.
Subtask identified: Take the action to Stay.
Calling function take_valid_action with parameters {'action': 'Stay'}
> {'output_1': 'State: Red, Result: Stayed Still'}



Observation: The current state is Light, and the last action taken was to stay still during a Red state.
Thoughts: Since the current state is Light, it is safe to move. To complete the assigned task of making a valid action, the next step should be to take the action "Move".
Subtask identified: Take the action "Move" as the current state is Light.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The game involves making valid actions based on the current state. Moving is allowed when the state is "Green" or "Light", and staying still is required when the state is "Red" or "Silence".
Thoughts: Since the current state is "Green" and the last action was moving forward, it is valid to move again.
Subtask identified: Take the action to move forward.
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Green, Result: Moved forward'}



Observation: The game involves making valid actions based on the current state. Moving is valid during "Light" or "Green" states, and staying still is valid during "Red" or "Silence" states. The current state is "Light" and the last action was moving forward.
Thoughts: Since the current state is "Light", it is valid to move forward. To complete the assigned task, I should take the action to move.
Subtask identified: Take the action to move forward as the current state is "Light".
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The current state is Silence, and the last action taken was moving forward. Moving during Silence results in elimination.
Thoughts: To make a valid action, I should stay still because the current state is Silence.
Subtask identified: Take the action to stay still.
Calling function take_valid_action with parameters {'action': 'Stay'}
> {'output_1': 'State: Silence, Result: Stayed Still'}



...

Observation: The game involves making valid actions based on the current state. Moving is allowed during "Light" or "Green" states, and staying still is required during "Red" or "Silence" states. The current state is "Light" and the last action was moving forward.
Thoughts: Since the current state is "Light", it is valid to move forward. To complete the assigned task, I should take the action to move.
Subtask identified: Take the action to move forward as the current state is "Light".
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The current state is Silence, and the last action taken was moving forward. Moving during Silence results in elimination.
Thoughts: To make a valid action, I should stay still since the current state is Silence.
Subtask identified: Take the action to stay still.
Calling function take_valid_action with parameters {'action': 'Stay'}
> {'output_1': 'State: Silence, Result: Stayed Still'}



Observation: The current state is Red, and the previous action was to stay still, which was the correct action.
Thoughts: Since the state is Red, the valid action is to stay still to avoid elimination. This aligns with the rules of the game.
Subtask identified: Take the action to stay still as the state is Red.
Calling function take_valid_action with parameters {'action': 'Stay'}
> {'output_1': 'State: Red, Result: Stayed Still'}



Observation: The game involves making valid actions based on the current state. Moving is allowed during "Light" or "Green" states, and staying still is required during "Red" or "Silence" states. The current state is "Light" and the last action was to stay still.
Thoughts: Since the current state is "Light", the valid action would be to move forward. This aligns with the rules of the game where moving is allowed during "Light" or "Green" states.
Subtask identified: Take the action to move forward as the current state is "Light".
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Light, Result: Moved forward'}



Observation: The game involves making valid actions based on the current state. Moving is allowed during "Light" or "Green" states, and staying still is required during "Red" or "Silence" states. The current state is "Green" and the last action was moving forward.
Thoughts: Since the current state is "Green", it is valid to move forward. To complete the assigned task, I should take the action to move.
Subtask identified: Take the action to move forward as the current state is "Green".
Calling function take_valid_action with parameters {'action': 'Move'}
> {'output_1': 'State: Green, Result: Success! Required moves met.'}

